[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](http://colab.research.google.com/github/ninarina12/ML_PNR/blob/master/pnr_properties.ipynb)

### Only if running in Google Colaboratory:
- Go to Runtime > Change runtime type, and select GPU.
- Clone the GitHub repository to access supplementary files:

In [ ]:
!git clone https://github.com/ninarina12/ML_PNR.git
%cd ML_PNR

### Visualize simulated and/or experimental PNR data

In [ ]:
%matplotlib inline
import numpy as np
import pandas as pd
import os
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt

from scipy.stats import mode
from utils.utils_data import u, parse_metadata, parse_labels, get_exp_names, read_exp
from utils.utils_plot import format_axis, fontsize, textsize, cmap, cmap_temp, FM_colors, AFM_colors

In [ ]:
# set data properties
sample_name = 'BiSe10_EuS5' #'CrO20_BiSbTe20'
dataset = 0

# get directories
set_dir = 'results/' + sample_name + '/set_' + str(dataset)
data_dir = set_dir + '/data'
image_dir = set_dir + '/properties'

if not os.path.exists(image_dir):
    os.makedirs(image_dir)

print('data:', set_dir)

In [ ]:
# data settings
layers, rho, M, N, q_min, q_max = parse_metadata(set_dir)
q = 10*np.linspace(q_min, q_max, N)

In [ ]:
# parse synthetic data
xdata_uu =  data_dir + '/xdata_uu.txt'
xdata_dd =  data_dir + '/xdata_dd.txt'
ydata =  data_dir + '/ydata.txt'

x_uu = pd.read_csv(xdata_uu, header=None, sep=' ', dtype=float).values
x_dd = pd.read_csv(xdata_dd, header=None, sep=' ', dtype=float).values
x = np.stack([x_uu, x_dd], axis=2)

df = pd.read_csv(ydata, dtype=float)
y_data, y_columns, y_header, y_ids, y_labels, y_units, y_units_ = parse_labels(df)

In [ ]:
# separate into classes if clear threshold present
df['class_prox'] = (df['magn_prox'] > 0.).astype(int)

# get ratio of classes
r = len(df.loc[df['class_prox'] == 1])/(len(df.loc[df['class_prox'] == 0]) + 1e-12)

# round to nearest multiple of 5
def nround(x, n=5):
    return n*round(x/n)
r = nround(r)
print(r)

In [ ]:
# organize dataframe of parameter values
layer = []
for l in layers:
    layer += [l]*len(df)

df = pd.DataFrame({'layer': layer,
                   'dens': np.concatenate([df[col].values for col in df.columns if col.startswith('dens_')]),
                   'd': np.concatenate([df[col].values for col in df.columns if col.startswith('d_')]),
                   's': np.concatenate([df[col].values for col in df.columns if col.startswith('s_')]),
                   'magn': np.concatenate([df[col].values for col in df.columns if col.startswith('magn_')]),
                   'class_prox': np.tile(df['class_prox'].values, len(layers))})

In [ ]:
# plot parameter ranges
u = 1.66054
df['M'] = df['layer'].map(lambda x: M[layers.index(x)])

# sampled density in g/cm^3
df['rho'] = df['dens']*u*df['M']/1e3

# magnetization in emu/cm^3
df['magn_'] = df['magn']*df['dens']*9.274

if sample_name == 'CrO20_BiSbTe20': sample_colors = AFM_colors
else: sample_colors = FM_colors
cols = dict(zip(layers, sample_colors))

fig, ((ax1, ax2, ax3), (ax4, ax5, ax6)) = plt.subplots(2,3, figsize=(11, len(layers)+1.5))
ylabels = ['t (nm)', r'$\rho\ (f.u./nm^3)$', r'$\rho\ (g/cm^3)$',
           r'$\sigma\ (nm)$', 'm ('+r'$\mu_B$/f.u.)', 'm ('+r'$emu/cm^3$)']

if r < 1e12:
    # split plot by class
    for i, (param, ax, ylabel) in enumerate(zip(['d', 'dens', 'rho', 's', 'magn', 'magn_'],
                                                [ax1, ax2, ax3, ax4, ax5, ax6], ylabels)):

        g = sns.violinplot(ax=ax, x=param, y='layer', hue='class_prox', split=True, inner=None, cut=0, width=0.6,
                           scale='count', palette={0:'w', 1:'k'}, data=df)
        g.legend_.remove()
        lg = len(g.collections)
        for j, item in enumerate(g.collections):
            c = int(np.round(item._paths[0].vertices[:,1].mean()))
            item._facecolors = np.array([mpl.colors.to_rgb(sample_colors[c]) + (1,)])

            if item._original_facecolor == (1.,1.,1.):
                item._hatch = "//"
                path = item._paths[0].vertices[:,1]
                b = mode(path)[0][0]
                item._paths[0].vertices[:,1][path != b] = b + r*(path[path != b] - b)
        
        ax.invert_yaxis()
        format_axis(ax, ylabel, '', nbins=4)
        if i%3: ax.set_yticklabels([])
    
    ax.legend(frameon=False, labels=['Pos. counts', r'Neg. counts $\times$ ' + str(r)],
              labelspacing=1., loc='lower left', bbox_to_anchor=(1,0))

else:
    for i, (param, ax, ylabel) in enumerate(zip(['d', 'dens', 'rho', 's', 'magn', 'magn_'],
                                                [ax1, ax2, ax3, ax4, ax5, ax6], ylabels)):

        g = sns.violinplot(ax=ax, x=param, y='layer', inner=None, cut=0, width=0.6,
                           scale='count', palette=cols, data=df)
        ax.invert_yaxis()
        format_axis(ax, ylabel, '', nbins=4)
        if i%3: ax.set_yticklabels([])
    
fig.tight_layout()
fig.savefig(image_dir + '/parameters.pdf')

### If experimental data available
Assumes experimental data files are stored in an `experiments/${SAMPLE}/` directory. Each measurement is stored in a sub-directory containing corresponding `x_uu.dat`, `x_dd.dat`, and `x_fit.csv`, where `x_fit.csv` contains the fitted spectra obtained by conventional methods.

In [ ]:
# plot experiment
exp_names = get_exp_names(sample_name)
labels = [k[:-1] + 'K' for k in exp_names]

fig, (ax1, ax2) = plt.subplots(1,2, figsize=(8.5,4.6), sharey=True)
temps = [int(k[:-1]) for k in exp_names]
norm = mpl.colors.Normalize(vmin=0, vmax=len(temps)-1)
for i in range(len(temps)):
    xdata_uu = 'experiments/' + sample_name + '/' + exp_names[i] + '/x_uu.dat'
    xdata_dd = 'experiments/' + sample_name + '/' + exp_names[i] + '/x_dd.dat'
    xdata_fit = 'experiments/' + sample_name + '/' + exp_names[i] + '/x_fit.csv'
    df_uu = read_exp(xdata_uu)
    df_dd = read_exp(xdata_dd)
    df_fit = pd.read_csv(xdata_fit)

    ax1.errorbar(10*df_uu['Q'], 10**i*df_uu['R'], 10**i*df_uu['dR'], lw=0, elinewidth=1, color=cmap_temp(norm(i)))
    ax2.errorbar(10*df_dd['Q'], 10**i*df_dd['R'], 10**i*df_dd['dR'], lw=0, elinewidth=1, color=cmap_temp(norm(i)))
    ax1.scatter(10*df_uu['Q'], 10**i*df_uu['R'], s=24, color=cmap_temp(norm(i)), ec='none', label=labels[i])
    ax2.scatter(10*df_dd['Q'], 10**i*df_dd['R'], s=24, color=cmap_temp(norm(i)), ec='none', label=labels[i])
    ax1.plot(10*df_fit['x'], 10**i*df_fit['y_uu'], color=cmap_temp(norm(i)), alpha=0.7)
    ax2.plot(10*df_fit['x'], 10**i*df_fit['y_dd'], color=cmap_temp(norm(i)), alpha=0.7)

ax1.set_yscale('log')
ax2.set_yscale('log')
ax1.legend(frameon=False, ncol=2, loc='upper right', columnspacing=0.2, handletextpad=0.2)
ax2.legend(frameon=False, ncol=2, loc='upper right', columnspacing=0.2, handletextpad=0.2)

format_axis(ax1, 'Q (nm$^{-1}$)', 'Reflectivity', 'R$^{++}$', xlims=[q.min(), q.max()])
format_axis(ax2, 'Q (nm$^{-1}$)', ' ', 'R$^{--}$', xlims=[q.min(), q.max()])

fig.tight_layout()
fig.subplots_adjust(wspace=0.1)
fig.savefig(image_dir + '/pnr_experiment.pdf')

In [ ]:
# save colorbar
cmap_temp_disc = mpl.colors.ListedColormap(cmap_temp(norm(range(len(temps)))))
norm = mpl.colors.Normalize(vmin=0, vmax=len(temps))
sm = mpl.cm.ScalarMappable(cmap=cmap_temp_disc, norm=norm)
sm.set_array([])

fig, ax = plt.subplots(figsize=(0.21,2.5))
cbar = fig.colorbar(sm, cax=ax, ticks=np.arange(0.5,len(temps)))
cbar.ax.set_yticklabels([str(i) for i in temps])
cbar.ax.minorticks_off()
format_axis(cbar.ax, '', '', 'T (K)')
fig.savefig(image_dir + '/pnr_experiment_cbar.pdf', bbox_inches='tight')